In [1]:
import pandas
import configparser
import psycopg2

In [2]:
config = configparser.ConfigParser()
config.read('configureir')


host = config['myaws']['host']
user = config['myaws']['user']
pswd = config['myaws']['pswd']
database = config['myaws']['database']

In [3]:
conn = psycopg2.connect (
                        host = host,
                        user = user,
                        password = pswd,
                        dbname = database
)

In [4]:
cur = conn.cursor()

In [5]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp14
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

In [6]:
cur.execute(table_sql)
conn.commit()

In [7]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=0'

In [8]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
print(html_data.decode('utf-8'))

<!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<meta http-equiv="content-type" content="text/html;charset=UTF-8">
<script type="text/javascript" src="/s/798993d/en_US.js"></script>
<script>window.indeed=window.indeed||{};indeed.crashtext=indeed.crashtext||{};indeed.crashtext.populate=indeed.crashtext.populate||function(c,g){var d=window[g];if(d){var e=d[""]||{},h=e.salt;if(e.hasOwnProperty("salt"))for(var b in c){for(var a=void 0,a=h,f=b.length;f;)a=33*a^b.charCodeAt(--f);a>>>=0;e.hasOwnProperty("id_length")&&(a=String(a).substring(0,e.id_length));d[a]=c[b]}else for(b in c)d[b]=[null].concat(c[b])}};indeed.crashtext.populate({"indeedapply_serp_label":["Easy apply"],"Turn on":["Subscribe"],"See new jobs for this search":["Get email alerts for this search"]}, 'indeed.i18n.localeData')</script>
<link href="/s/60d7c47/jobsearch_all.css" rel="stylesheet" type="text/css">
<link rel="alternate" type="application/rss+xml" title="Intelligence Analyst Jobs, Employment" href="http://rss.indeed

In [9]:
!pip install beautifulsoup4

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
print (soup)

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<script src="/s/798993d/en_US.js" type="text/javascript"></script>
<script>window.indeed=window.indeed||{};indeed.crashtext=indeed.crashtext||{};indeed.crashtext.populate=indeed.crashtext.populate||function(c,g){var d=window[g];if(d){var e=d[""]||{},h=e.salt;if(e.hasOwnProperty("salt"))for(var b in c){for(var a=void 0,a=h,f=b.length;f;)a=33*a^b.charCodeAt(--f);a>>>=0;e.hasOwnProperty("id_length")&&(a=String(a).substring(0,e.id_length));d[a]=c[b]}else for(b in c)d[b]=[null].concat(c[b])}};indeed.crashtext.populate({"indeedapply_serp_label":["Easy apply"],"Turn on":["Subscribe"],"See new jobs for this search":["Get email alerts for this search"]}, 'indeed.i18n.localeData')</script>
<link href="/s/60d7c47/jobsearch_all.css" rel="stylesheet" type="text/css"/>
<link href="http://rss.indeed.com/rss?q=intelligence+analyst" rel="alternate" title="Intelligence Analyst Jobs, Emp

In [11]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [12]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)


In [13]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

In [16]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp14(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

In [18]:
df = pandas.read_sql_query('select * from indeed_gp14', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Intelligence Analyst,National Security Agency,"Fort Meade, MD","$47,016 - $74,759 a year",The National Security Agency (NSA) is part of ...
1,2,Quantitative Analyst / Developer,RT Signal,"New York, NY 10013 (Tribeca area)","$95,000 - $165,000 a year","We are a small, but well-funded (financially s..."
2,3,H-ISAC Cyber Threat Intelligence Analyst,Perch Security,"Titusville, FL","$50,000 - $75,000 a year",Ability to communicate technical issues to tec...
3,4,Security Analyst,Seen by Indeed,"Washington, DC",None,Get matched – Companies apply to you with oppo...
4,5,"Cybersecurity Analyst (DLP) (60-70k, US Citize...",Maveris,"Martinsburg, WV","$60,000 - $70,000 a year",The Analyst can identify risks in security sys...
5,6,Transportation Analyst,Lipman Family Farms,"Immokalee, FL 34142",None,"Perform fact based, data driven analysis, and ..."
6,7,ERP Applications Developer (Epicor),"Ashworth Bros., Inc.","Winchester, VA 22601",None,This may include custom mobile and windows app...
7,8,Intelligence Analyst,Booz Allen Hamilton,None,None,We’re an EOE that empowers our people—no matte...
8,9,Intelligence Analyst I - MRS,WorldAware,None,None,WorldAware is dedicated to equal opportunity i...
9,10,Intelligence Analyst,BAE Systems,None,None,Having an understanding of physical and techni...


In [19]:
cur.close()
conn.close()